In [1]:
print("Hi")
print("Wo?")

Hi
Wo?


In [ ]:
import numpy as np
import pandas as pd
from math import radians, sin, cos, sqrt, atan2
import heapq
from collections import defaultdict
import matplotlib.pyplot as plt

# Load data
node_coords = np.loadtxt('./nodepl.txt')
node_list = np.loadtxt('./nodelist.txt', dtype=int)
arc_data = pd.read_csv('./arclist.txt', delim_whitespace=True, header=None,
                       names=["destination", "time", "distance", "speed_limit", "clazz", "flags"])
arc_data['time'] *= 3600  # Convert time to seconds

# Haversine function to calculate distance between two points
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Earth radius in km
    dlat = radians(lat2 - lat1)
    dlon = radians(lon2 - lon1)
    a = sin(dlat / 2) ** 2 + cos(radians(lat1)) * cos(radians(lat2)) * sin(dlon / 2) ** 2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    return R * c

# Set home coordinates and find nearest node
home_lat, home_lon = 47.065307430925586, 15.44306321533678
distances = [haversine(home_lat, home_lon, lat, lon) for lat, lon in node_coords]
start_node = np.argmin(distances) + 1  # 1-based node ID

# Build graph
graph = defaultdict(list)
for index, row in arc_data.iterrows():
    if index >= len(node_list):  # Skip if index is out of bounds for node_list
        continue
    source = node_list[index] - 1  # Convert 1-based index to 0-based for internal use
    destination, time, distance = row['destination'] - 1, row['time'], row['distance']
    graph[source].append((destination, time, distance, row['clazz'], row['flags']))

def dijkstra(graph, start, target, cost_type='time'):
    queue = [(0, start)]
    # Initialize distances with all nodes in graph as integer keys
    distances = {int(node): float('inf') for node in graph}
    distances[start] = 0
    predecessors = {start: None}

    while queue:
        current_distance, current_node = heapq.heappop(queue)

        if current_node == target:
            break

        if current_distance > distances[current_node]:
            continue

        for neighbor, time, distance, clazz, flags in graph[current_node]:
            neighbor = int(neighbor)  # Ensure neighbor is an integer key
            if neighbor not in distances:
                distances[neighbor] = float('inf')  # Add missing neighbor with default distance

            # Choose cost type
            cost = time if cost_type == 'time' else distance
            distance_via_current = current_distance + cost

            # Update shortest path
            if distance_via_current < distances[neighbor]:
                distances[neighbor] = distance_via_current
                predecessors[neighbor] = current_node
                heapq.heappush(queue, (distance_via_current, neighbor))

    return distances, predecessors


# Custom cost function (weighted combination of time and distance)
def custom_cost(row):
    time_weight, distance_weight = 0.5, 0.5
    return time_weight * row['time'] + distance_weight * row['distance']

# Dijkstra’s algorithm with custom cost
def dijkstra_custom(graph, start, target):
    queue = [(0, start)]
    distances = {node: float('inf') for node in graph}
    distances[start] = 0
    predecessors = {start: None}

    while queue:
        current_distance, current_node = heapq.heappop(queue)
        if current_node == target:
            break

        for neighbor, time, distance, clazz, flags in graph[current_node]:
            cost = custom_cost({'time': time, 'distance': distance})
            distance_via_current = current_distance + cost
            if distance_via_current < distances[neighbor]:
                distances[neighbor] = distance_via_current
                predecessors[neighbor] = current_node
                heapq.heappush(queue, (distance_via_current, neighbor))

    return distances, predecessors

# Plotting function for routes
def plot_route(predecessors, start, destination):
    route = []
    node = destination
    while node is not None:
        route.append(node)
        node = predecessors.get(node)
    route = route[::-1]

    lats, lons = zip(*[node_coords[node] for node in route])
    plt.plot(lons, lats, marker='o')
    plt.xlabel("Longitude")
    plt.ylabel("Latitude")
    plt.title(f"Route from {start} to {destination}")
    plt.show()


# Destinations in Graz
destinations = [9328, 6031, 8543]
results = {}

# Berechnung der Distanzen für jedes Ziel
for dest in destinations:
    distances, predecessors = dijkstra(graph, start_node - 1, dest - 1, cost_type='time')
    results[dest] = {
        'time': (distances, predecessors),
        'distance': dijkstra(graph, start_node - 1, dest - 1, cost_type='distance'),
        'custom': dijkstra_custom(graph, start_node - 1, dest - 1)
    }

    # Prüfen, ob der Zielknoten in den Distanzen vorhanden ist
    if dest - 1 in distances:  # 0-basierter Index
        print(f"Shortest Distance to node {dest}: {distances[dest - 1]} units")
    else:
        print(f"Node {dest} is not reachable.")

# Plot and compare routes for each destination
for dest in destinations:
    print(f"\nRoutes to destination node {dest}:")
    for cost_type, (distances, predecessors) in results[dest].items():
        print(f"\nCost Type: {cost_type.capitalize()}")

        # Ausgabe der kürzesten Distanz zum Ziel
        if dest - 1 in distances:
            print(f"Shortest Distance: {distances[dest - 1]} units")
        else:
            print(f"Node {dest} is not reachable.")

        plot_route(predecessors, start_node, dest)
